## Imports

In [1]:
# %matplotlib notebook

# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from xgboost import XGBRegressor

import os
import pickle
import wordcloud as wc
import scipy.sparse as sparse
import xgboost as xgb
import random
import jenkspy
import statsmodels.tools.tools as stattools
from itertools import combinations

import sklearn
from sklearn import tree
from sklearn.model_selection import KFold,train_test_split
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [2]:
Dataset_path = "./DataSets/"
Movies_metadata = pd.read_csv(Dataset_path+"MoviesInfo.csv")
ratings = pd.read_csv(Dataset_path+"CleanedRatings.csv")

In [3]:
# then merge movie and rating data
data = pd.merge(Movies_metadata[['movieId', 'title']] ,ratings)

In [4]:
print( f"number of movies : { ratings.groupby('movieId').count().shape[0] }")
print( f"number of users : { ratings.groupby('userId').count().shape[0] }")
print( f"number of ratins : { ratings.shape[0] }")
print( f"range of rating : ( { np.min( ratings.groupby('rating').count().index )}, {np.max( ratings.groupby('rating').count().index )})  ")

number of movies : 70
number of users : 6032
number of ratins : 171457
range of rating : ( 0.5, 5.0)  


In [5]:
# data = data.iloc[:1000000,:]

In [6]:
pivot_table = data.pivot_table(index = ["userId"],columns = ["title"],values = "rating")
pivot_table.head(10)

title,A spasso con Daisy,Ace Ventura - Missione Africa,American Graffiti,Angeli e insetti,Arlington Road - L'inganno,Assassins,"Babe, maialino coraggioso",Betty Love,Camera con vista,Casinò,...,Stigmate,Stripes - Un plotone di svitati,The Players Club,Tonto + tonto,Toy Story - Il mondo dei giocattoli,Tre colori - Film blu,Un giorno di ordinaria follia,Un lupo mannaro americano a Londra,Via da Las Vegas,Weekend con il morto
userId,,,,,,,,,,,,,,,,,,,,,
4,NaN,2.0,NaN,4.0,NaN,3.0,3.5,NaN,NaN,4.0,...,3.5,NaN,NaN,0.5,4.0,NaN,3.5,NaN,4.5,1.0
56,NaN,0.5,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,2.5,4.0,NaN,3.0,NaN,NaN
100,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,4.5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,NaN,4.0,4.0,3.0,NaN,NaN,4.0,...,4.0,NaN,NaN,NaN,5.0,NaN,3.0,NaN,3.0,NaN
134,NaN,3.0,4.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,3.0,4.5,NaN,NaN,5.0,NaN,NaN,3.5,NaN,5.0
173,NaN,2.0,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN
214,3.0,2.0,NaN,3.0,3.0,NaN,1.0,4.0,NaN,4.0,...,NaN,NaN,NaN,NaN,3.0,4.0,4.0,NaN,3.0,NaN
235,3.0,3.0,5.0,NaN,4.0,NaN,5.0,3.0,NaN,4.0,...,NaN,NaN,NaN,1.0,5.0,NaN,3.0,4.0,NaN,NaN
239,3.0,1.0,3.0,NaN,2.0,NaN,4.0,3.0,NaN,4.0,...,4.0,3.0,NaN,1.0,3.0,4.0,3.0,5.0,NaN,3.0


In [7]:
def Simple_Item_Based_Collaborative_Filtering(FilmName):
    movie_watched = pivot_table[FilmName]
    
    # find correlation between "FilmName" and other movies
    similarity_with_other_movies = pivot_table.corrwith(movie_watched) 
    
    similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
    
    display( similarity_with_other_movies.head() )

In [8]:
Simple_Item_Based_Collaborative_Filtering("Jumanji")

title
Jumanji                         1.000000
The Players Club                0.487965
La prossima vittima             0.463108
La strada dei quartieri alti    0.458677
Pensieri pericolosi             0.433583
dtype: float64

In [9]:
AvailableMovies =  pivot_table.columns.to_list()

Simple_Item_Based_Collaborative_Filtering( random.choice(AvailableMovies) )

title
La strada dei quartieri alti               1.000000
The Players Club                           0.866025
Infernale avventura (L')                   0.697515
Tonto + tonto                              0.500614
Le straordinarie avventure di Pinocchio    0.498970
dtype: float64